In [212]:
from string import ascii_letters

import pandas as pd

from utils.paths import BOOKS_PATH, AUTHORS_PATH

In [213]:
books_df = pd.read_csv(BOOKS_PATH)
authors_df = pd.read_csv(AUTHORS_PATH)

/tmp/ipykernel_102037/587830805.py:1: DtypeWarning: Columns (76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv(BOOKS_PATH)


In [214]:
books_df['author_href']

0           https://lubimyczytac.pl/autor/3701/j-k-rowling
1           https://lubimyczytac.pl/autor/3701/j-k-rowling
2           https://lubimyczytac.pl/autor/3701/j-k-rowling
3           https://lubimyczytac.pl/autor/3701/j-k-rowling
4        https://lubimyczytac.pl/autor/14276/antoine-de...
                               ...                        
15493        https://lubimyczytac.pl/autor/21483/sara-gran
15494    https://lubimyczytac.pl/autor/32829/jean-paul-...
15495       https://lubimyczytac.pl/autor/3282/david-weber
15496    https://lubimyczytac.pl/autor/22983/katarzyna-...
15497      https://lubimyczytac.pl/autor/6389/martin-booth
Name: author_href, Length: 15498, dtype: object

In [215]:
authors_df['author_url']

0          https://lubimyczytac.pl/autor/3701/j-k-rowling
1       https://lubimyczytac.pl/autor/14276/antoine-de...
2        https://lubimyczytac.pl/autor/3216/j-r-r-tolkien
3       https://lubimyczytac.pl/autor/10474/george-orwell
4       https://lubimyczytac.pl/autor/22701/michail-bu...
                              ...                        
5003    https://lubimyczytac.pl/autor/36392/dmitrij-st...
5004    https://lubimyczytac.pl/autor/4018/slawomir-ce...
5005    https://lubimyczytac.pl/autor/30318/marcin-bar...
5006    https://lubimyczytac.pl/autor/32747/cathi-hanauer
5007        https://lubimyczytac.pl/autor/21483/sara-gran
Name: author_url, Length: 5008, dtype: object

In [216]:
data_df = books_df.join(authors_df.set_index('author_url'), on='author_href', how='inner')

In [217]:
# check if cycles always end with <cycle name> (<string> <number>)
data_df['Cykl'].dropna().str.extract(r'(\d)+').fillna(0).astype(int)[0].unique()

array([1, 2, 3, 4, 5, 6, 7, 0, 8, 9])

In [218]:
data_df['part_of_cycle'] = data_df['Cykl'].str.extract(r'(\d)+').fillna(0).astype(int)[0]

In [219]:
data_df['part_of_cycle'].sort_values(ascending=False)[:3]

15449    9
1860     9
1835     9
Name: part_of_cycle, dtype: int64

In [220]:
data_df.loc[data_df['part_of_cycle'].sort_values(ascending=False)[:3].index, :]

,author,author_href,description,number_of_user_opinions,number_of_user_ratings,number_of_discussions,Kategoria,Format,Cykl,Tytuł oryginału,...,author_href27,author_name,author_average_rating,author_number_of_people_read,author_number_of_people_wants_to_read,author_date_of_birth,author_number_of_fans,author_number_of_books_written,author_number_of_awards,part_of_cycle
15449,Cindy Jefferies,https://lubimyczytac.pl/autor/6162/cindy-jeffe...,"Najsłynniejsze uczennice Rockley Park, bliźni...",2,142,0,literatura dziecięca,papier,Szkoła Gwiazd (tom 9),Fame School: Pop Diva,...,NaN,Cindy Jefferies,6.0,928,564,Unknown,12,20,0,9
1860,Terry Pratchett,https://lubimyczytac.pl/autor/3221/terry-pratc...,"Powszechnie uznawana prawda głosi, że kiedy p...",151,2556,0,"fantasy, science fiction",papier,Świat Dysku (tom 39),Snuff,...,NaN,Terry Pratchett,7.4,36760,107286,28.04.1948,5401,166,0,9
1835,Agatha Christie,https://lubimyczytac.pl/autor/14083/agatha-chr...,Stara pani Boynton nie należy do czułych mate...,155,2575,0,"kryminał, sensacja, thriller",papier,Herkules Poirot (tom 19),Appointment with Death,...,NaN,Agatha Christie,7.0,58293,132793,15.09.1890,6416,186,0,9


In [221]:
# drop records without number of pages
data_df = data_df.dropna(subset=['Liczba stron'])

In [222]:
numeric_columns = data_df.select_dtypes(include='number').columns

In [223]:
float_columns = 'author_average_rating'
integer_columns = numeric_columns.drop(float_columns)

In [224]:
# integer cols where there is nans
data_df.loc[:, integer_columns] = data_df.loc[:, integer_columns].fillna(0).astype(int)

In [225]:
data_df[integer_columns]

,number_of_user_opinions,number_of_user_ratings,number_of_discussions,Liczba stron,number_of_people_read,number_of_people_has,number_of_people_favorite,number_of_people_wants_to_read,number_of_people_wants_as_gift,number_of_people_currently_read,...,rating_4,rating_3,rating_2,rating_1,author_number_of_people_read,author_number_of_people_wants_to_read,author_number_of_fans,author_number_of_books_written,author_number_of_awards,part_of_cycle
0,5156,85009,1,320.0,124692,30249.0,11410.0,6210,990.0,849.0,...,501,490,67,145,130174,75668,11479,40,1,1
1,2305,70818,0,356.0,105466,25003.0,8972.0,4535,883.0,501.0,...,345,404,26,67,130174,75668,11479,40,1,2
2,2241,67918,0,456.0,101297,23587.0,10242.0,5041,938.0,463.0,...,200,235,20,46,130174,75668,11479,40,1,3
3,1892,64439,0,764.0,96528,22705.0,9109.0,5211,971.0,632.0,...,244,274,27,55,130174,75668,11479,40,1,4
4,4264,63437,0,144.0,91987,16084.0,7102.0,3781,536.0,312.0,...,1040,1139,335,428,87296,8410,681,42,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15493,22,149,0,272.0,210,22.0,6.0,145,0.0,0.0,...,12,6,6,5,323,230,1,3,0,0
15494,9,147,0,773.0,223,52.0,17.0,810,7.0,26.0,...,4,2,1,1,2886,8533,274,37,0,0
15495,13,153,0,314.0,236,62.0,9.0,91,3.0,0.0,...,5,5,1,3,1717,6447,215,85,0,1
15496,11,152,0,224.0,248,46.0,3.0,57,0.0,0.0,...,8,3,1,1,1952,1233,21,9,0,0


# imputacja tytulow

In [226]:
data_df.columns

Index(['author', 'author_href', 'description', 'number_of_user_opinions',
       'number_of_user_ratings', 'number_of_discussions', 'Kategoria',
       'Format', 'Cykl', 'Tytuł oryginału',
       ...
       'author_href27', 'author_name', 'author_average_rating',
       'author_number_of_people_read', 'author_number_of_people_wants_to_read',
       'author_date_of_birth', 'author_number_of_fans',
       'author_number_of_books_written', 'author_number_of_awards',
       'part_of_cycle'],
      dtype='object', length=103)

In [227]:
missing_title = data_df['Tytuł oryginału'].isna()

In [228]:
data_df.loc[missing_title, 'Tytuł oryginału'] = data_df.loc[missing_title, 'url'].str.split('/').str[-1].str.split('-').str.join(' ').str.title()

In [229]:
data_df['part_of_cycle']

0        1
1        2
2        3
3        4
4        0
        ..
15493    0
15494    0
15495    1
15496    0
15497    2
Name: part_of_cycle, Length: 15186, dtype: int64

# cleaning

In [230]:
data_df.columns

Index(['author', 'author_href', 'description', 'number_of_user_opinions',
       'number_of_user_ratings', 'number_of_discussions', 'Kategoria',
       'Format', 'Cykl', 'Tytuł oryginału',
       ...
       'author_href27', 'author_name', 'author_average_rating',
       'author_number_of_people_read', 'author_number_of_people_wants_to_read',
       'author_date_of_birth', 'author_number_of_fans',
       'author_number_of_books_written', 'author_number_of_awards',
       'part_of_cycle'],
      dtype='object', length=103)

## author quantitative features:
author_average_rating
author_number_of_people_read
author_number_of_people_wants_to_read
author_date_of_birth ?
author_number_of_fans
author_number_of_books_written
author_number_of_awards
## book quantitative features
description_length (engineered)
number_of_user_opinions
number_of_user_ratings
number_of_discussions
number_of_people_read
Data wydania
Liczba stron
description
## book qualitative features
Kategoria
part_of_cycle (engineered)
Język
number_of_people_has
number_of_people_favorite
number_of_people_wants_to_read
number_of_people_wants_as_gift
number_of_people_currently_read

In [231]:
author_quant_features = [
    "author_average_rating",
    "author_number_of_people_read",
    "author_number_of_people_wants_to_read",
    "author_date_of_birth",  # Możliwe, że wymaga dodatkowego przetworzenia
    "author_number_of_fans",
    "author_number_of_books_written",
    "author_number_of_awards"
]

# Cechy ilościowe związane z książką
book_quant_features = [
    "number_of_user_opinions",
    "number_of_user_ratings",
    "number_of_discussions",
    "number_of_people_read",
    "Data wydania",  # to be transformed
    "Liczba stron",
    "Cykl",
] + [f'rating_{i}' for i in range(1, 11)]

# Cechy jakościowe związane z książką
book_qual_features = [
    "Kategoria",
    "part_of_cycle",  # engineered
    "Język",
    "number_of_people_has",
    "number_of_people_favorite",
    "number_of_people_wants_to_read",
    "number_of_people_wants_as_gift",
    "number_of_people_currently_read",
    "description",
    "Format"
]

In [232]:
all_features = author_quant_features + book_quant_features + book_qual_features

In [233]:
data_df[all_features].columns

Index(['author_average_rating', 'author_number_of_people_read',
       'author_number_of_people_wants_to_read', 'author_date_of_birth',
       'author_number_of_fans', 'author_number_of_books_written',
       'author_number_of_awards', 'number_of_user_opinions',
       'number_of_user_ratings', 'number_of_discussions',
       'number_of_people_read', 'Data wydania', 'Liczba stron', 'Cykl',
       'rating_1', 'rating_2', 'rating_3', 'rating_4', 'rating_5', 'rating_6',
       'rating_7', 'rating_8', 'rating_9', 'rating_10', 'Kategoria',
       'part_of_cycle', 'Język', 'number_of_people_has',
       'number_of_people_favorite', 'number_of_people_wants_to_read',
       'number_of_people_wants_as_gift', 'number_of_people_currently_read',
       'description', 'Format'],
      dtype='object')

In [234]:
data_df.loc[:, 'author_date_of_birth'] = pd.to_datetime(data_df['author_date_of_birth'], format='mixed', errors='coerce')

In [235]:
data_df.loc[:, 'author_date_of_birth']

0        1965-07-31 00:00:00
1        1965-07-31 00:00:00
2        1965-07-31 00:00:00
3        1965-07-31 00:00:00
4        1900-06-29 00:00:00
                ...         
15493                    NaT
15494    1905-06-21 00:00:00
15495    1952-10-24 00:00:00
15496    1963-01-01 00:00:00
15497                    NaT
Name: author_date_of_birth, Length: 15186, dtype: object

In [236]:
data_df[all_features].columns

Index(['author_average_rating', 'author_number_of_people_read',
       'author_number_of_people_wants_to_read', 'author_date_of_birth',
       'author_number_of_fans', 'author_number_of_books_written',
       'author_number_of_awards', 'number_of_user_opinions',
       'number_of_user_ratings', 'number_of_discussions',
       'number_of_people_read', 'Data wydania', 'Liczba stron', 'Cykl',
       'rating_1', 'rating_2', 'rating_3', 'rating_4', 'rating_5', 'rating_6',
       'rating_7', 'rating_8', 'rating_9', 'rating_10', 'Kategoria',
       'part_of_cycle', 'Język', 'number_of_people_has',
       'number_of_people_favorite', 'number_of_people_wants_to_read',
       'number_of_people_wants_as_gift', 'number_of_people_currently_read',
       'description', 'Format'],
      dtype='object')

In [237]:
rename_dict = {
    "Data wydania": "publication_date",
    "Liczba stron": "number_of_pages",
    "Cykl": "series",
    "Kategoria": "category",
    "Język": "language",
    "Format": "format"
}

In [238]:
renamed_cols = [rename_dict.get(col, col) for col in data_df[all_features].columns]

In [239]:
renamed_cols

['author_average_rating',
 'author_number_of_people_read',
 'author_number_of_people_wants_to_read',
 'author_date_of_birth',
 'author_number_of_fans',
 'author_number_of_books_written',
 'author_number_of_awards',
 'number_of_user_opinions',
 'number_of_user_ratings',
 'number_of_discussions',
 'number_of_people_read',
 'publication_date',
 'number_of_pages',
 'series',
 'rating_1',
 'rating_2',
 'rating_3',
 'rating_4',
 'rating_5',
 'rating_6',
 'rating_7',
 'rating_8',
 'rating_9',
 'rating_10',
 'category',
 'part_of_cycle',
 'language',
 'number_of_people_has',
 'number_of_people_favorite',
 'number_of_people_wants_to_read',
 'number_of_people_wants_as_gift',
 'number_of_people_currently_read',
 'description',
 'format']

In [240]:
# data_df = data_df.rename(columns=rename_dict)
data_df = data_df.rename(columns=rename_dict)

In [241]:
data_df[renamed_cols].to_csv('data/cleaned_data.csv')